In [5]:
import numpy as np
from bs4 import BeautifulSoup
import tensorflow as tf
import urllib
import pickle

In [3]:
class MP:
    def __init__(self, name, twitter_handle, party):
        self.name = name;
        self.twitter_handle = twitter_handle
        self.party = party
        self.tweets = []
    def addTweet(self, tweet):
        self.tweets.append(tweet)
    def __str__(self):
        return 'Name: %s, TW: %s, Party: %s' % (self.name, self.twitter_handle, self.party)
        
list_of_mps = []

url = 'https://www.mpsontwitter.co.uk/list'
with urllib.request.urlopen(url) as doc:
    soup = BeautifulSoup(doc, 'html.parser')
    rows = soup.find_all('tr')
    for i in range(2,len(rows)):
        party = rows[i].find('td')['class'][0]
        twitter_handle = rows[i].find('a').getText()
        name = rows[i].find_all('td')[2].getText()
        list_of_mps.append(MP(name, twitter_handle, party))
        

In [103]:
consumer_key = 'MVb3kWys0DQXRBmcWWkyE7Gbz'
consumer_secret = 'xUr49m74r9wNYWu0JJWMvfMQ0sDCrISaFyjJlPqu0JS2J7hRmf'
access_token = '1239937231-A3hMA2n7CWtQjdozut3bUqhUcNMK9L1FvGepLX1'
access_secret = 'lDETflnQ8khU4iIisPaixcVNe0a7dLKKwVgWNDBzhvFEe'


import tweepy as tw
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tw.API(auth)

for mp in list_of_mps:
    try:
        tweets = api.user_timeline(mp.twitter_handle, count=50)
        for tweet in tweets:
            mp.addTweet(tweet.text)
    except tw.TweepError:
        print('Protected Tweets :(')
        
with open('test.txt', 'wb') as testFile:
    pickle.dump(list_of_mps, testFile)

Protected Tweets :(
Protected Tweets :(
Protected Tweets :(
Protected Tweets :(
Protected Tweets :(
Protected Tweets :(
Protected Tweets :(


In [70]:
from nltk.tokenize import TweetTokenizer
import math
tknzr = TweetTokenizer()

with open('data.txt', 'rb') as testFile:
    mps = pickle.load(testFile)

In [57]:
test = 'the quick brown fox jumped over the lazy dog'
t = test.split()
def skip_gram(dataset, vocabulary, window_size=2):
    pairs = []
    for (idx, word) in enumerate(dataset):
        if word not in vocabulary:
            vocabulary[word] = 1
        if idx >= window_size and idx < len(dataset)-window_size:
            for i in range(idx - window_size, idx + window_size + 1):
                if idx != i:
                    pairs.append((dataset[idx], dataset[i]))
    return pairs

dataset = []
vocabulary = {}

for mp in mps:
    if len(mp.tweets) > 0:
        mp_skip_gram = []
        for tweet in mp.tweets:
            tweet_list = tknzr.tokenize(tweet)
            mp_skip_gram += skip_gram(tweet_list, vocabulary)
        dataset += mp_skip_gram
        

In [60]:
vocabulary_size = len(vocabulary)
dataset_size = len(dataset)
embedding_size = 128

In [71]:
embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
nce_weights = tf.Variable(
    tf.truncated_normal([vocabulary_size, embedding_size],
                        stddev=1.0/math.sqrt(embedding_size)))
nce_biases = tf.Variable(tf.zeros([vocabulary_size]))